# **RAG model - NLP - házi feladat**
AX83OL

## Rövid ismertető

A következő colbertv2.0 ragatouille előtanított model a PTE valamint a PTE EHÖK hivatalos szabályzatain lett továbbtanítva.

Az alábbi weboldalak alatt lévő dokumentumokat használja:

*   A Pécsi Tudományegyetem Szervezeti és Működési Szabályzata és mellékletei:

    https://adminisztracio.pte.hu/adminisztracio/szabalyzatok_utasitasok/szervezeti_es_mukodesi_szabalyzat_es_mellekletek

*   PTE EHÖK Dokumentumtár:

    https://pteehok.hu/dokumentumtar/

Az összes dokumentum manuálisan le lett töltve a két oldalról, majd feltöltve ZIP fájlként a Google Drive-ra, onnan tud ez a notebook hozzáférni.

A továbbtanításhoz használni kívánt PDF-ek szerencsére szövegként tartalmaznak mindent, így azok felhasználhatóak voltak. (Illetve volt talán 2-3 melléklet ami nem, de azok amúgy sem túl kardinális témáról szólnak, így azokat kihagytam.)

Az összes használt doksi (=tanítóhalmaz) együttvéve kb. 1800 oldalt tesz ki.

# MEGVALÓSÍTÁS:

Feltelepítjük a két alapvetően szükséges modult (CUDA-hoz és RAG-hoz kell).

In [ ]:
!pip install optimum auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.0/410.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
!pip install --ignore-installed ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 866.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.0/493.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 751.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 3.

Betöltjük az adott modelt.

In [ ]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

Process ForkProcess-18:
Process ForkProcess-25:
Process ForkProcess-32:
Process ForkProcess-12:
Process ForkProcess-22:
Process ForkProcess-20:
Process ForkProcess-31:
Process ForkProcess-14:
Process ForkProcess-13:
Process ForkProcess-21:
Process ForkProcess-3:
Process ForkProcess-15:
Process ForkProcess-11:
Process ForkProcess-4:
Process ForkProcess-23:
Process ForkProcess-24:
Process ForkProcess-9:
Process ForkProcess-17:
Process ForkProcess-29:
Process ForkProcess-8:
Process ForkProcess-27:
Process ForkProcess-10:
Process ForkProcess-26:
Process ForkProcess-28:
Process ForkProcess-19:
Process ForkProcess-30:
Process ForkProcess-16:
Process ForkProcess-1:
Process ForkProcess-2:
Process ForkProcess-7:
Process ForkProcess-6:
Process ForkProcess-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call las

KeyboardInterrupt: 

Kell majd a PyPDF2 modul a PDF-ek feldolgozásához, így azt is feltelepítjük.

In [ ]:
!pip install PyPDF2

Készítünk egy függvényt ami a Google Drive-ról letöltött és már kicsomagolt mappában lévő összes PDF doksin végigmegy, majd azokat betölti egy string változóba.

In [ ]:
import os
from PyPDF2 import PdfReader

def read_pdfs(directory):
    """
    Read all PDF files in the specified directory and concatenate their contents into a single string.

    :param directory: str - Path to the directory containing PDF files.
    :return: str - Concatenated text content of all PDF files.
    """
    full_text = ""

    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)

        # Check if the file is a PDF
        if filepath.endswith(".pdf"):
            with open(filepath, "rb") as file:
                reader = PdfReader(file)

                # Iterate through each page of the PDF
                for page_num in range(len(reader.pages)):
                    page = reader.pages[page_num]
                    full_text += page.extract_text()

    return full_text

Letöltjük a forrásadatokat.

In [ ]:
!gdown 16Pn9bYZSY3no-QXvwKlsK0ORrdaG0TQX

Kicsomagoljuk a letöltött ZIP-et, hogy alkalmazni tudjuk az így kapott mappára a read_pdfs függvényt.

In [ ]:
import zipfile

# A ZIP fájl elérési útja
zip_file_path = "Szabalyzatok.zip"

# A célkönyvtár, ahova kicsomagoljuk a fájlokat
extract_to_directory = ""

# Megnyitjuk a ZIP fájlt és kicsomagoljuk a tartalmát
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

print("A ZIP fájl sikeresen kicsomagolva.")

Alkalmazzuk az előbb említett függvényt.

In [ ]:
# Directory containing the PDF files
pdf_directory = "Szabalyzatok"

# Read all PDF files in the directory and concatenate their contents
full_document = read_pdfs(pdf_directory)

# Check the length of the concatenated text
print(len(full_document))
type(full_document)

In [ ]:
full_document

A tanítóhalmaz indexeléséhez eltávolítjuk a ragatoullie modullal automatikusan installált CPU modult, majd meggyőződünk, hogy a GPU-s verzió fent van-e.

In [ ]:
!pip uninstall faiss-cpu
!pip install faiss-gpu

### Indexeljük a tanítóhalmazt, itt hossznak 180-at adtam meg: ez azt jelenti, hogy milyen hosszú részekre darabolja fel a tanítóhalmazt. => Majd a válaszok milyen hosszúak legyenek max.

In [ ]:
RAG.index(collection=[full_document], index_name="PTE", max_document_length=180, split_documents=True)

A következő függvénnyel tudunk kérdéseket feltenni a megtanult doksikkal kapcsolatban.

In [ ]:
def inputFunction(question):
  k = 3 # How many documents you want to retrieve, defaults to 10, we set it to 3 here for readability
  results = RAG.search(query=question, k=k)
  return results[0]['content'].replace("\n","").replace("   ","  ").replace("  "," ")

Tesztként:

In [ ]:
result = inputFunction("Mikor utasítja el érdemi vizsgálat nékül az EHÖK a kollégiumi jelentkezésemet?")
print(result)

## Betöltünk a Hugging Face-ről egy transformers szöveggeneráló modelt amely támogat magyar nyelvet (és remélhetőleg elbírja a Colab rendszere is). Majd átadjuk ennek az előző colbertv2.0 RAG model válaszát.

In [ ]:
from transformers import pipeline

def theFunction(theQuestion):

  answerSkeleton = inputFunction(theQuestion)

  prompt = f"{theQuestion} válaszold meg a következő szöveg alapján, csak a szöveget használhatod fel: {answerSkeleton}"
  generator = pipeline(task="text-generation", model="NYTK/PULI-GPT-2", max_length=500)

  return generator(prompt)[0]["generated_text"]

In [ ]:
theFunction("Mikor utasítja el érdemi vizsgálat nékül az EHÖK a kollégiumi jelentkezésemet?")

A Gradio-t is fel kell telepítenünk a webes GUI miatt.

In [ ]:
!pip install gradio

Megvalósítunk egy egyszerű webes GUI-t:

In [ ]:
import gradio as gr

# Létrehozunk egy Gradio interfészt a függvényünkkel
interface = gr.Interface(
    fn=theFunction,
    inputs="text",
    outputs="text",
    title="RAG Model Summary Generator (PTE admin edition)",
    description="Generates a summary for a given input text using the RAG model. This RAG model trained on many administrative documents of PTE and PTE EHÖK.",
)

# A Gradio interfész indítása
interface.launch()